STARTED:      1/29/25

LAST UPDATED: 1/29/25

By Eugene Fong

# TUTORIAL - ANNDATA

SOURCE: [https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html](https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html)

### IMPORTS

In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix
print(ad.__version__)

### INITIALIZING ANNDATA

Let’s start by building a basic AnnData object with some sparse count information, perhaps representing gene expression counts.

In [ ]:
counts = csr_matrix(np.random.poisson(1, size=(100, 2000)), dtype=np.float32)
adata = ad.AnnData(counts)
adata

We see that AnnData provides a representation with summary stastics of the data The initial data we passed are accessible as a sparse matrix using adata.X.

In [ ]:
adata.X

Now, we provide the index to both the `obs` and `var` axes using `.obs_names` (resp. `.var_names`).

In [ ]:
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
print(adata.obs_names[:10])

### Subsetting AnnData

These index values can be used to subset the AnnData, which provides a view of the AnnData object. We can imagine this to be useful to subset the AnnData to particular cell types or gene modules of interest. The rules for subsetting AnnData are quite similar to that of a Pandas DataFrame. You can use values in the `obs/var_names`, boolean masks, or cell index integers.

In [ ]:
adata[["Cell_1", "Cell_10"], ["Gene_5", "Gene_1900"]]

### Adding aligned metadata

### Observation/Variable level

So we have the core of our object and now we’d like to add metadata at both the observation and variable levels. This is pretty simple with AnnData, both `adata.obs` and `adata.var` are Pandas DataFrames.

In [ ]:
ct = np.random.choice(["B", "T", "Monocyte"], size=(adata.n_obs,))
adata.obs["cell_type"] = pd.Categorical(ct)  # Categoricals are preferred for efficiency
adata.obs

We can also see now that the AnnData representation has been updated:

In [ ]:
adata

### Subsetting using metadata

We can also subset the AnnData using these randomly generated cell types:

In [ ]:
bdata = adata[adata.obs.cell_type == "B"]
bdata

# Observation/variable-level matrices

We might also have metadata at either level that has many dimensions to it, such as a UMAP embedding of the data. For this type of metadata, AnnData has the `.obsm/.varm` attributes. We use keys to identify the different matrices we insert. The restriction of `.obsm/.varm` are that .obsm matrices must length equal to the number of observations as `.n_obs` and `.varm` matrices must length equal to `.n_vars`. They can each independently have different number of dimensions.

Let’s start with a randomly generated matrix that we can interpret as a UMAP embedding of the data we’d like to store, as well as some random gene-level metadata:

In [ ]:
adata.obsm["X_umap"] = np.random.normal(0, 1, size=(adata.n_obs, 2))
adata.varm["gene_stuff"] = np.random.normal(0, 1, size=(adata.n_vars, 5))
adata.obsm

Again, the AnnData representation is updated.

In [ ]:
adata

A few more notes about `.obsm/.varm`

The “array-like” metadata can originate from a Pandas DataFrame, scipy sparse matrix, or numpy dense array.

When using scanpy, their values (columns) are not easily plotted, where instead items from `.obs` are easily plotted on, e.g., UMAP plots.

### Unstructured metadata

AnnData has `.uns`, which allows for any unstructured metadata. This can be anything, like a list or a dictionary with some general information that was useful in the analysis of our data.

In [ ]:
adata.uns["random"] = [1, 2, 3]
adata.uns

### Layers

Finally, we may have different forms of our original core data, perhaps one that is normalized and one that is not. These can be stored in different layers in AnnData. For example, let’s log transform the original data and store it in a layer:

In [ ]:
adata.layers["log_transformed"] = np.log1p(adata.X)
adata

### Conversion to DataFrames

We can also ask AnnData to return us a DataFrame from one of the layers:

In [ ]:
adata.to_df(layer="log_transformed")

We see that the `.obs_names/.var_names` are used in the creation of this Pandas object.

### Writing the results to disk

AnnData comes with its own persistent HDF5-based file format: `h5ad`. If string columns with small number of categories aren’t yet categoricals, `AnnData` will auto-transform to categoricals.

In [14]:
adata.write('my_results.h5ad', compression="gzip")

### Wrapping up the introduction
AnnData has become the standard for single-cell analysis in Python and for good reason – it’s straightforward to use and faciliatates more reproducible analyses with it’s key-based storage. It’s even becoming easier to convert to the popular R-based formats for single-cell analysis.

Keep reading on to better understand “views”, on-disk backing, and other details.

### Views and copies
For the fun of it, let’s look at another metadata use case. Imagine that the observations come from instruments characterizing 10 readouts in a multi-year study with samples taken from different subjects at different sites. We’d typically get that information in some format and then store it in a DataFrame:

In [16]:
obs_meta = pd.DataFrame({
        'time_yr': np.random.choice([0, 2, 4, 8], adata.n_obs),
        'subject_id': np.random.choice(['subject 1', 'subject 2', 'subject 4', 'subject 8'], adata.n_obs),
        'instrument_type': np.random.choice(['type a', 'type b'], adata.n_obs),
        'site': np.random.choice(['site x', 'site y'], adata.n_obs),
    },
    index=adata.obs.index,    # these are the same IDs of observations as above!
)

This is how we join the readout data with the metadata. Of course, the first argument of the following call for `X` could also just be a DataFrame.

In [17]:
adata = ad.AnnData(adata.X, obs=obs_meta, var=adata.var)

Now we again have a single data container that keeps track of everything.

In [ ]:
print(adata)

Similar to numpy arrays, AnnData objects can either hold actual data or reference another `AnnData` object. In the later case, they are referred to as “view”.

Subsetting AnnData objects always returns views, which has two advantages:

- no new memory is allocated
- it is possible to modify the underlying AnnData object

You can get an actual AnnData object from a view by calling `.copy()` on the view. Usually, this is not necessary, as any modification of elements of a view (calling .[] on an attribute of the view) internally calls `.copy()` and makes the view an AnnData object that holds actual data. See the example below.

In [ ]:
adata

Indexing into AnnData will assume that integer arguments to `[]` behave like `.iloc` in pandas, whereas string arguments behave like `.loc`. AnnData always assumes string indices.

In [ ]:
adata[:5, ['Gene_1', 'Gene_3']]

This is a view! If we want an `AnnData` that holds the data in memory, let’s call `.copy()`

In [21]:
adata_subset = adata[:5, ['Gene_1', 'Gene_3']].copy()

For a view, we can also set the first 3 elements of a column.

In [ ]:
print(adata[:3, 'Gene_1'].X.toarray().tolist())
adata[:3, 'Gene_1'].X = [0, 0, 0]
print(adata[:3, 'Gene_1'].X.toarray().tolist())

If you try to access parts of a view of an AnnData, the content will be auto-copied and a data-storing object will be generated.

In [ ]:
adata_subset = adata[:3, ['Gene_1', 'Gene_2']]
adata_subset

In [ ]:
adata_subset.obs['foo'] = range(3)

Now adata_subset stores the actual data and is no longer just a reference to adata.

In [ ]:
adata_subset

Evidently, you can use all of pandas to slice with sequences or boolean indices.

In [ ]:
adata[adata.obs.time_yr.isin([2, 4])].obs.head()

### Partial reading of large data

If a single `.h5ad` is very large, you can partially read it into memory by using backed mode:

In [29]:
adata = ad.read('my_results.h5ad', backed='r')

In [ ]:
adata.isbacked

If you do this, you’ll need to remember that the `AnnData` object has an open connection to the file used for reading:

In [ ]:
adata.filename

As we’re using it in read-only mode, we can’t damage anything. To proceed with this tutorial, we still need to explicitly close it:

In [32]:
adata.file.close()

As usual, you should rather use `with` statements to avoid dangling open files (up-coming feature).

Manipulating the object on disk is possible, but experimental for sparse data. Hence, we leave it out of this tutorial.